In [5]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib import style
import seaborn as sns
import datetime as dt
import ipywidgets as w
import ipympl
sns.set_theme()
style.use('ggplot')

pd.set_option('display.max_rows', 1000)


In [6]:
def dividendo_por_acao(lista, data):
    dividendo = lista[(lista.index.month == data.month) & (lista.index.year == data.year)]
    dividendo = dividendo.sum()
    return dividendo


def calcula_investimento(ticker, inicio='2010-01-01', termino='2020-01-01', aporte_mensal=1000.0):
    lista_de_dividendos = yf.Ticker(ticker).dividends
    hist = yf.download(ticker, start=inicio, end=termino, interval='1mo')
    hist = hist.dropna()
    hist['valor_investido'] = 0.00
    hist['aporte_mensal'] = aporte_mensal
    hist['aporte_total'] = 0.00
    hist['total_acoes'] = 0.00
    hist['resto'] = 0.00
    hist['dividendos'] = 0.00
    total_de_linhas = hist.shape[0]

    for i in range(0, total_de_linhas):
        linha = hist.iloc[i,:]
        data = hist.index[i]
        valor_de_fechamento = linha['Adj Close']
        if i > 0:
            linha_anterior = hist.iloc[i-1, :]
            acoes_anterior = linha_anterior['total_acoes']
            resto_anterior = linha_anterior['resto']
        else:
            # i == 0
            acoes_anterior = 0.00
            resto_anterior = 0.00
            
        dividendo = dividendo_por_acao(lista_de_dividendos, data)
        total_dividendos = dividendo*acoes_anterior
        aporte_total = aporte_mensal + resto_anterior + total_dividendos
        novas_acoes = int(aporte_total/valor_de_fechamento)
        total_acoes = acoes_anterior + novas_acoes
        valor_investido = total_acoes*valor_de_fechamento
        resto = aporte_total - novas_acoes*valor_de_fechamento
        # substituir valores
        hist.at[data, 'valor_investido'] = valor_investido
        hist.at[data, 'aporte_total'] = aporte_total
        hist.at[data, 'total_acoes'] = total_acoes
        hist.at[data, 'resto'] = resto
        hist.at[data, 'dividendos'] = total_dividendos

    return hist

In [7]:
carteira = [
    {
        "ticker":"ITUB3.SA",
        "inicio":"2010-01-01",
        "termino":"2020-10-10",
        "aporte_mensal":200.0
    },
    {
        "ticker":"ABEV3.SA",
        "inicio":"2010-01-01",
        "termino":"2020-10-10",
        "aporte_mensal":300.0
    },
    {
        "ticker":"B3SA3.SA",
        "inicio":"2010-01-01",
        "termino":"2020-10-10",
        "aporte_mensal":300.0
    },
    {
        "ticker":"WEGE3.SA",
        "inicio":"2010-01-01",
        "termino":"2020-10-10",
        "aporte_mensal":300.0
    },
    {
        "ticker":"EGIE3.SA",
        "inicio":"2010-01-01",
        "termino":"2020-10-10",
        "aporte_mensal":200.0
    }
]

In [8]:
def calcula_carteira(portfolio):
    COLS = ['data', 'ticker', 'valor_investido', 'aporte_mensal', 'aporte_total', 'total_acoes', 'dividendos']
    resultado = pd.DataFrame(columns=COLS)
    if len(portfolio) > 0:
        for acao in portfolio:
            try:
                hist = calcula_investimento(
                    ticker=acao['ticker'], inicio=acao['inicio'], termino=acao['termino'], aporte_mensal=acao['aporte_mensal']
                    )
                print('Ativo calculado!')
                hist = hist.reset_index()
                hist = hist.rename(columns={'Date':'data'})
                hist['ticker'] = acao['ticker']
                resultado = resultado.append(hist[COLS], ignore_index=True)
            except:
                print('Não foi possível calcular o ticker: ', acao['ticker'])
                pass
        return resultado
    else:
        print('A carteira precisa ter pelo menos 1 ativo.')

In [9]:
def cria_grafico(resultado):
    fig, ((ax1, ax2),(ax3, ax4)) = plt.subplots(2,2, figsize=(25, 15))
    fig.suptitle('Performance da carteira', size=28, weight='bold')

    formato_moeda = ticker.StrMethodFormatter('R${x:,.0f}')
    ax1.yaxis.set_major_formatter(formato_moeda)
    ax2.yaxis.set_major_formatter(formato_moeda)
    ax3.yaxis.set_major_formatter(formato_moeda)
    ax4.yaxis.set_major_formatter(formato_moeda)

    ax1.set_title('Total investido em R$')
    ax2.set_title('Total Aportado por Ação')
    ax3.set_title('Dividendos Acumulados')
    ax4.set_title('Total de dividendos recebidos')

    # pivot tables
    pivot_aportes = resultado.pivot(index='data', columns='ticker', values='aporte_mensal').fillna(0)
    pivot_dividendos = resultado.pivot(index='data', columns='ticker', values='dividendos').fillna(0)
    pivot_valor_investido = resultado.pivot(index='data', columns='ticker', values='valor_investido').fillna(0)
    pivot_dividendo_acumulado = pivot_dividendos.cumsum()

    pivot_aportes.plot.area(ax=ax2, stacked=True)
    pivot_valor_investido.plot.area(ax=ax1, stacked=True)
    pivot_dividendo_acumulado.plot.area(ax=ax3, stacked=True)
    pivot_dividendos.plot.line(ax=ax4, stacked=False)

    return fig

## ITERFACE GRAFICA

In [20]:
def cria_carteira(lista_tickers, aporte, inicio, termino):
    carteira = []
    for ticker in lista_tickers:
        acao = {
            'ticker': ticker.strip(),
            'inicio': inicio,
            'termino': termino,
            'aporte_mensal': aporte
        }
        carteira.append(acao)
    
    return carteira


def processar(b):
    with output:
        lista_tickers = tickers.value.split(',')
        aporte_mensal = int(aporte_slider.value)
        inicio = data_inicio.value
        termino = data_termino.value
        carteira = cria_carteira(lista_tickers, aporte_mensal, inicio, termino)
        df = calcula_carteira(carteira)
        figura = cria_grafico(df)
        output.clear_output()
        display(figura)


output = w.Output()

header = w.HTML("<h1>Calculadora de Investimentos</h1>", layout=w.Layout(height='auto'))

tickers = w.Text(placeholder='Ex: ITUB3.SA, ABEV3.SA,...')
ticker_label = w.Label('Entre com tickers separados por vírgula.')

data_inicio = w.DatePicker()
data_inicio_label = w.Label('Escolha data inicial dos investimentos:')

data_termino = w.DatePicker()
data_termino_label = w.Label('Escolha data final dos investimentos:')

aporte_texto = w.FloatText()
aporte_slider = w.FloatSlider(min=0, max=10000, step=100)
aporte_label = w.Label('Selecione o aporte mensal:')

w.jslink((aporte_texto, 'value'), (aporte_slider,'value'))

botao_submit = w.Button(description='Calcular', layout={'padding': '0px 20px', 'margin':'20px 5px'}, button_style='success', icon='play')
botao_submit.on_click(processar)


menu = w.VBox([ticker_label, tickers, data_inicio_label, data_inicio, data_termino_label, data_termino, aporte_label, aporte_texto, aporte_slider, botao_submit], layout=w.Layout(padding='20px'))

w.AppLayout(header=header, left_sidebar=menu, center=output, right_sidebar=None, footer=None, height='1000px', grid_gap='20px')

AppLayout(children=(HTML(value='<h1>Calculadora de Investimentos</h1>', layout=Layout(grid_area='header', heig…